In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [4]:
loader = PDFPlumberLoader("data/diabetes draft7.pdf")
docs = loader.load()

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
documents = text_splitter.split_documents(docs)

In [13]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [14]:
bm25_retriever = BM25Retriever.from_documents(documents)

In [16]:
vector_store = FAISS.from_documents(documents, embeddings)
faiss_retriever = vector_store.as_retriever()

In [17]:
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, faiss_retriever], weight={0.4, 0.6})

In [18]:
ensemble_retriever

EnsembleRetriever(retrievers=[BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x000001F197991550>), VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001F198C6E6D0>)], weights=[0.5, 0.5])

In [21]:
template = """
    당신은 친절한 ai 어시스턴트입니다. 질문에 주어진 context를 활용해 자세히 답변해 주세요.
    답변을 할때, 출처를 반드시 남겨주세요.
    만약, 답변내용을 모른다면, 모른다고 잡해주세요.
    답변은 한국어로 답변해 주세요.

    #Question:
    {question}
    #Context:
    {context}

    #Answer:
"""

prompt = PromptTemplate.from_template(template=template)

In [22]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [23]:
chain = ({"context":ensemble_retriever, "question":RunnablePassthrough()}
         |prompt
         |llm
         |StrOutputParser()
         )

In [24]:
chain.invoke("vif에 대해서 알려줘")

'VIF(Variance Inflation Factor)는 회귀 분석에서 다중 공선성(multicollinearity)을 진단하는 데 사용되는 지표입니다. 다중 공선성이란 독립 변수 간에 상관관계가 높아져서 회귀 계수의 추정치가 불안정하게 되는 현상을 의미합니다. VIF는 특정 독립 변수가 다른 독립 변수들로 설명되는 정도를 나타내며, 다음과 같이 계산됩니다.\n\n\\[ \\text{VIF}_i = \\frac{1}{1 - R_i^2} \\]\n\n여기서 \\( R_i^2 \\)는 i번째 독립 변수를 다른 모든 독립 변수들로 회귀 분석했을 때의 결정계수입니다. VIF 값이 1에 가까울수록 해당 독립 변수는 다른 변수와의 상관관계가 적음을 의미하며, 일반적으로 VIF 값이 10을 초과하면 다중 공선성이 존재한다고 간주합니다.\n\nVIF를 통해 다중 공선성 문제를 발견하면, 해당 변수를 제거하거나 다른 방법(예: 주성분 분석)을 통해 문제를 해결할 수 있습니다.\n\n출처: [Document에서 제공된 내용](data/diabetes draft7.pdf)'